<a href="https://colab.research.google.com/github/lawgorithm/large_scale_inference/blob/main/Tests_for_many_independent_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
# from scipy.stats import t
# from scipy.stats import norm
import statsmodels.formula.api as smf


# Simulate observed data

In [2]:
def generate_normal_normal_samples(n_samples, mu, tau2):
    """
    Generates samples from a Normal-Normal hierarchical model.  V is simulated
    sampled from an Inverse Gamma distribution.

    Args:
        n_samples (int): The number of samples to generate.
        mu (float): The prior mean of theta.
        tau2 (float): The prior variance of theta.

    Returns:
        tuple: A tuple containing:
            - theta (array-like): The generated theta values.
            - y (array-like): The generated y values.
            - V (array-like): The generated V values (observation variances).
    """

    # Generate theta values from the prior distribution
    theta = np.random.normal(mu, np.sqrt(tau2), n_samples)

    # Simulate V values from an Inverse Gamma distribution (example prior)
    alpha = 5  # Shape parameter for the Inverse Gamma
    beta = 5   # Scale parameter for the Inverse Gamma
    V = 1.0 / np.random.gamma(alpha, 1.0 / beta, n_samples)

    # Generate y values from the conditional distribution (vectorized)
    y = np.random.normal(theta, np.sqrt(V))

    return theta, y, V

In [3]:
# Example usage:
n_samples = 100
mu = 0
tau2 = 2

theta, y, V = generate_normal_normal_samples(n_samples, mu, tau2)

df = pd.DataFrame({
    "true_mean": theta,
    "y": y,
    "true_SD": V**0.5
})
df['experiment'] = range(len(df['true_mean']))
df['experiment']= df['experiment'].astype(str)

NUM_BUCKETS = 19
for index, row in df.iterrows():
  samples = np.random.normal(
    loc=row['true_mean'],
    scale=row['true_SD'],
    size=19
  )
  df_tmp = pd.DataFrame({
    "y": samples,
  })
  df_tmp['true_mean'] = row['true_mean']
  df_tmp['true_SD'] = row['true_SD']
  df_tmp['experiment'] = row['experiment']
  df = pd.concat([df, df_tmp], ignore_index=True)

df = df.sort_values(by=['experiment']).reset_index(drop=True)
df



,true_mean,y,true_SD,experiment
0,-2.983310,-2.237498,0.885909,0
1,-2.983310,-3.416547,0.885909,0
2,-2.983310,-2.142861,0.885909,0
3,-2.983310,-2.366834,0.885909,0
4,-2.983310,-3.469896,0.885909,0
...,...,...,...,...
1995,-0.480427,0.764505,1.401126,99
1996,-0.480427,-2.011671,1.401126,99
1997,-0.480427,0.359882,1.401126,99
1998,-0.480427,-1.188932,1.401126,99


In [4]:
# df_agg = df.groupby(['true_mean', 'true_SD'], as_index=False).agg(
#     y_bar = ('y', 'mean'),
#     SE = ('y', 'std'))
df_agg = df.groupby(['experiment'], as_index=False).agg(
    y_bar = ('y', 'mean'),
    SE = ('y', 'std')).sort_values(by=['experiment'])
df_agg['DoF'] = 19
df_agg['t_stat'] = df_agg['y_bar'] / df_agg['SE']
df_agg.head()

,experiment,y_bar,SE,DoF,t_stat
0,0,-3.166691,0.978545,19,-3.236121
1,1,0.339858,0.776514,19,0.437672
2,10,0.984978,3.191718,19,0.308604
3,11,-1.750995,0.773178,19,-2.264673
4,12,-1.601219,1.051505,19,-1.522788


# Methods that use aggregated summaries like t-stats or p-values

## Fisher's Method

**Overview**

Convert each t-statistic to a p-value (using the appropriate degrees of freedom) and then compute:

$$ X = -2 \sum_{i=1}^k \log{p_i} $$

Under the global null, follows a chi-square distribution with
2 * k degrees of freedom. This approach is powerful when the tests are independent

**Global Null (TODO)**

wikipedia say's fisher's method is "is used to combine the results from several independence tests bearing upon the same overall hypothesis (H0)" -- that doesn't seem to be the case here. I am not replicating the same test over and over (in which case they would have the same null) -- rather, I am running many different independent tests. Is that a problem?

Fisher's method is designed to combine p‑values from independent tests that all address the same overall null hypothesis. In your case, even though you're running different tests (one in each zip code), if each test's null hypothesis is “no effect in that zip code” then collectively they address the global null hypothesis “no zip code has an effect.” That means you're effectively testing the same hypothesis in different independent settings.

So, even though the tests come from different locations, as long as:

Each test is carried out correctly (yielding valid p‑values under its own null), and
The tests are independent (or approximately so),
Fisher's method (or other p‑value combination methods) is applicable. The key is that all tests are assessing the same overall hypothesis (zero effect everywhere), even if the tests themselves differ in power or sample size.

If the tests were addressing fundamentally different null hypotheses, then combining them with Fisher's method wouldn’t be appropriate. But here, since the underlying null is conceptually the same, using Fisher's method is valid.

**References**

* [Asymptotic Optimality of Fisher's Method of Combining Independent Tests](https://www.tandfonline.com/doi/abs/10.1080/01621459.1971.10482347)
* [Powerful p-value combination methods to detect incomplete association](https://www.nature.com/articles/s41598-021-86465-y)

**Independence Assumption**

Fisher's method combines p-values by assuming that they are independent. In genetic studies—and other high-dimensional applications—the tests are often correlated. For many parallel, independent experiments, it should be fine.

**Limitations**

* While Fisher's method is perfectly valid under independence, it can be overly influenced by a few very small p-values. Fisher's method sums the logarithms of p-values, so a few very small p-values can dominate the test statistic.

In [5]:
def fisher_combined_pvalue(t_stats, dof):
    """
    Combine p-values from independent t-tests using Fisher's method.

    Parameters:
      t_stats : array_like
          Array of t-statistics from individual experiments.
      dof : int or array_like
          Degrees of freedom for each t-statistic (can be scalar if same for all).

    Returns:
      X : float
          The Fisher combined test statistic.
      p_value : float
          Combined p-value under the global null.
    """
    # Convert each t-statistic to a two-sided p-value.
    t_stats = np.asarray(t_stats)
    if np.isscalar(dof):
        dof = np.full(t_stats.shape, dof)
    p_values = 2 * (1 - stats.t.cdf(np.abs(t_stats), dof))

    # Fisher's method statistic
    X = -2 * np.sum(np.log(p_values))

    # Under the null, X ~ chi2 with 2*k degrees of freedom, where k = number of tests.
    k = len(t_stats)
    p_value = 1 - stats.chi2.cdf(X, 2*k)
    return X, p_value

In [6]:
#@title Check behavior for null effects

np.random.seed(42)
n_experiments = 1000
# suppose each experiment has 20 observations
dof = 19

# Generate 1000 t-statistics under the global null (no true effect).
# Under H0, t-statistics follow a t-distribution.
t_stats_null = np.random.standard_t(dof, size=n_experiments)

X, combined_p = fisher_combined_pvalue(t_stats_null, dof)
print("Fisher combined statistic:", X)
print("Combined p-value:", combined_p)

Fisher combined statistic: 1983.4123943800714
Combined p-value: 0.5996546079628109


In [7]:
#@title Check behavior for non-null simulated data

X, combined_p = fisher_combined_pvalue(
    t_stats=df_agg['t_stat'],
    dof=df_agg['DoF'])
print("Fisher combined statistic:", X)
print("Combined p-value:", combined_p)

Fisher combined statistic: 321.44768430729
Combined p-value: 1.0858032739591295e-07


## Stouffer's Method

Stouffer’s Method (Weighted Z‐Test):
First, each independent t-statistic to an equivilent p-value. Then, use the normal density to convert each p-value to a z-score. Then combine them by summing (possibly with weights) and standardizing. Because the tests are independent, the sum (appropriately normalized) will be approximately standard normal under the global null. This method is less dominated by a few extremely small p‑values compared to Fisher’s method and is quite natural when many tests are weak.

Here you convert each test’s p‑value to a z‑score (or directly convert the t‑statistic if the degrees of freedom are similar across tests), then combine them:

$$Z_{\text{combined}} = \frac{\sum_{i=1}^p N_i}{\sqrt{p}}$$

where each $Z_i$ is the z-score corresponding to the i-th test. Under the global null,

$$Z_{\text{combined}} \sim N(0,1) $$

This method can be more robust when the signals are weak and spread out, and you can also apply weights if some tests are more precise than others.

In [8]:
def stouffer_combined_pvalue(t_stats, dof):
    """
    Combine p-values from independent t-tests using Stouffer's method (Z-score combination).

    Parameters:
      t_stats : array_like
          Array of t-statistics from individual experiments.
      df : int or array_like
          Degrees of freedom for each t-statistic (assumed identical here for simplicity).

    Returns:
      Z_combined : float
          The combined Z-score.
      p_value : float
          Combined p-value under the global null.
    """
    t_stats = np.asarray(t_stats)
    if np.isscalar(dof):
        dof = np.full(t_stats.shape, dof)
    # Convert to two-sided p-values and then to Z-scores
    p_values = 2 * (1 - stats.t.cdf(np.abs(t_stats), dof))
    # For a two-sided test, convert p-values to Z-scores:
    Z_scores = stats.norm.ppf(1 - p_values/2)
    # Restore the sign of t-statistics
    Z_scores = np.sign(t_stats) * Z_scores
    # Combine Z-scores; if tests are equally weighted:
    Z_combined = np.sum(Z_scores) / np.sqrt(len(Z_scores))
    combined_p = 2 * (1 - stats.norm.cdf(np.abs(Z_combined)))
    return Z_combined, combined_p

In [9]:
#@title Check behavior for null effects

Z_combined, combined_p_stouffer = stouffer_combined_pvalue(
    t_stats=t_stats_null,
    dof=dof)
print("Stouffer combined Z:", Z_combined)
print("Combined p-value:", combined_p_stouffer)

Stouffer combined Z: -1.5182621845214257
Combined p-value: 0.12894831667914253


In [10]:
#@title Check behavior for non-null simulated data

Z_combined, combined_p_stouffer = stouffer_combined_pvalue(
    t_stats=df_agg['t_stat'],
    dof=df_agg['DoF']
)
print("Stouffer combined Z:", Z_combined)
print("Combined p-value:", combined_p_stouffer)

Stouffer combined Z: -2.738527779072329
Combined p-value: 0.006171494642797537


# Methods that use the raw data

## Why use the raw data rather than combine t-stats, z-scores or p-values?

Imagine the following thought experiment: suppose you replicate the same experiment twice. Is it more powerful to compute a p-value for each and then use fisher's method (or a similar method that aggregates p-values or t-statistics)? Or should you combine all the data and do a sinlge test?

When the same experiment is replicated under identical conditions, combining all the data into a single test is usually more powerful than analyzing each replication separately and then combining the p-values (e.g., via Fisher's method).

Combining the raw data yields a larger overall sample size, which improves the precision of estimates and increases the test's power. When you run separate tests and then combine p-values, you lose some information about the effect sizes and their variability. A single test that pools the data uses all available degrees of freedom, whereas separate tests have fewer degrees of freedom each. Pooling data also typically results in a more reliable estimate of the common variance.

Methods like Fisher's method are designed for situations where you might not have access to the raw data (for example, if studies are published separately) or if the studies differ in design. When the experiments are identical, a joint analysis is both simpler and more statistically efficient.

## ANOVA


In this case you do have individual-level observations for each experiment, you can treat “experiment_id” as a categorical factor in a regression model or ANOVA. For example, you might fit a model like

$$y_{ij} = \mu + \alpha_j + \epsilon_{ij}$$

where $\alpha_j$ represents the deviation for each experiment.

Here the global null hypothesis is

$$ H_0: \alpha_1 = ... = \alpha_j = 0 $$

meaning all experiments have the same mean. This is a standard ANOVA problem, and you can test it with an F-test.

Below, we do this by building a weighted least squares regression (using weighted least squares to handle heterskedasticity of the error terms in the model above).

We can then either:
*   Use a likelihood ratio test to test the significance of the categorical variable (equivilent to testing whether $\alpha_1 = ... = \alpha_j = 0)$ via LRT
*   Directly use an ANOVA F-test

**References**

* [ANOVA F-test](https://en.wikipedia.org/wiki/F-test#One-way_analysis_of_variance)
* ANOVA in Casella and Berger

In [11]:
# In this approach, weights are inversely proportional to each experiment's
# sampling variance.

df_anova = df.merge(
    df_agg[['experiment', 'SE']],
    how='left',
    on=['experiment']).copy()
df_anova = df_anova[['experiment', 'y', 'SE']]
df_anova['weights'] = 1 / df_anova['SE']**2

# Fit full WLS model
wls_full_model = smf.wls(
    "y ~ C(experiment)",
    data=df_anova,
    weights=df_anova['weights']
    ).fit()

# Fit the reduced WLS model (without experiment)
wls_reduced_model = smf.wls(
    "y ~ 1",  # Intercept-only model
    data=df_anova,
    weights=df_anova['weights']
).fit()

# Note: the F-statistic printed here is the same as the F-stats printed below
print(wls_full_model.summary())

                            WLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.702
Model:                            WLS   Adj. R-squared:                  0.687
Method:                 Least Squares   F-statistic:                     45.24
Date:                Sun, 23 Feb 2025   Prob (F-statistic):               0.00
Time:                        20:28:15   Log-Likelihood:                -2846.6
No. Observations:                2000   AIC:                             5893.
Df Residuals:                    1900   BIC:                             6453.
Df Model:                          99                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              -3.1667    

In [12]:
#@title Likelihood ratio test testing significance of categorical variable
lr_stat = 2 * (wls_full_model.llf - wls_reduced_model.llf)
df_diff = wls_full_model.df_model - wls_reduced_model.df_model
p_value = 1 - stats.chi2.cdf(lr_stat, df_diff) # check whether this is correct test for LRT
print(f"LR Statistic: {lr_stat}")
print(f"p-value: {p_value}")

LR Statistic: 2422.268386652835
p-value: 0.0


In [13]:
#@title Direct F-test testing significance of categorical variable
f_test = wls_full_model.compare_f_test(wls_reduced_model)
print(f"F statistic: {f_test[0]}, p-value: {f_test[1]}")

F statistic: 45.24092514582824, p-value: 0.0


In [14]:
#@title Recreation of F-test

# Calculate WSS
wss_full = wls_full_model.ssr
wss_reduced = wls_reduced_model.ssr

# Degrees of freedom
df_full = wls_full_model.df_resid
df_reduced = wls_reduced_model.df_resid

# "F-like" statistic
f_statistic = ((wss_reduced - wss_full) / (df_reduced - df_full)) / (wss_full / df_full)

# P-value (using F-distribution)
p_value = 1 - stats.f.cdf(f_statistic, df_reduced - df_full, df_full) # Need scipy for this

print(f"WSS Full Model: {wss_full}")
print(f"WSS Reduced Model: {wss_reduced}")
print(f"F-statistic: {f_statistic}")
print(f"P-value: {p_value}")

WSS Full Model: 1900.0
WSS Reduced Model: 6378.851589436996
F-statistic: 45.24092514582824
P-value: 1.1102230246251565e-16


In [15]:
assert False

AssertionError: 

## Mixed Effects (frequentist)

Interesting. Suppose instead i replicate the same experiment twice. Is it more powerful to compute a pvalur for each and then use fisher’s method? Or combine all the data and do a sinlge test?

When the same experiment is replicated under identical conditions, combining all the data into a single test is usually more powerful than analyzing each replication separately and then combining the p‑values (e.g., via Fisher’s method). Here’s why:

Efficiency and Information Use:
Combining the raw data yields a larger overall sample size, which improves the precision of estimates and increases the test’s power. When you run separate tests and then combine p‑values, you lose some information about the effect sizes and their variability.
Degrees of Freedom:
A single test that pools the data uses all available degrees of freedom, whereas separate tests have fewer degrees of freedom each. Pooling data typically results in a more reliable estimate of the common variance.
Meta-Analysis vs. Joint Analysis:
Methods like Fisher’s method are designed for situations where you might not have access to the raw data (for example, if studies are published separately) or if the studies differ in design. When the experiments are identical, a joint analysis is both simpler and more statistically efficient.
In summary, if the experiments are identical and you have access to all the raw data, it is generally more powerful to combine all the data and perform a single, unified test.

Interesting. That suggests to me that, even in the independent tests in different zip codes example, i might be better off with a different method that starts with thr raw data rather than starting with t-stats or pvalues

Yes, exactly. When you have access to the raw data from each zip code, a hierarchical (multilevel) model is often more powerful and efficient than analyzing each zip code separately and then combining p‑values. Here's why:

Borrowing Strength: A hierarchical model pools information across zip codes. Even if each zip code provides only a weak signal individually, the model can "borrow strength" across the data, leading to more precise estimates of the overall effect and the variability among zip codes.
Handling Heterogeneity: A hierarchical model can incorporate both within–zip code variability and between–zip code variability. This is particularly important if zip codes vary in sample sizes or if the treatment effect might differ slightly from one area to another.
Direct Estimation of the Global Effect: Instead of combining summary statistics (like t‑stats or p‑values), a joint analysis of the raw data allows you to directly test whether there is an overall nonzero effect while accounting for the structure of the data.
Flexibility: You can include covariates, random effects, or even allow for spatial correlation if that’s relevant. This flexibility can improve power and control for confounding factors.
For example, if you have individual-level outcomes from all zip codes, you might model them as follows (in a mixed-effects framework):

$$y_{ij} = \mu + \tau_j + \epsilon_{ij}$$

where $y_{ij}$ is the outcome for unit i in experiment j. $\mu$ is the overall mean effect (the parameter of interest), $\tau_j$ is the random effect for each experiment. We assume:

$$\tau_j \stackrel{\text{iid}}{\sim} N(0, \sigma^2_j)$$

$$\epsilon_{ij} \stackrel{\text{iid}}{\sim} N(0, \sigma^2)$$

In Python, packages such as statsmodels (for frequentist approaches) or PyMC3/NumPyro (for Bayesian approaches) allow you to fit such models. This approach makes full use of the raw data and is typically more powerful than combining p-values from separate tests.

Alternatively, if you view the zip codes as a random sample from a larger population, you might use a random effects model:


Here, testing whether the zip codes add value means testing whether
σ
τ
2
>
0
σ
τ
2
​
 >0. This is typically done via a likelihood ratio test, with the appropriate mixture of chi‑squares because of the boundary issue

 n a random effects model, the null hypothesis is that
σ
τ
2
=
0
σ
τ
2
​
 =0. Even though the model is written with a common fixed effect
μ
μ, the variation among groups is captured in
σ
τ
2
σ
τ
2
​
 . Testing
H
0
:
σ
τ
2
=
0
H
0
​
 :σ
τ
2
​
 =0 tells you whether there is any evidence that the zip codes differ.

 Random Effects vs. Fixed Effects
Random Effects Model:
In a standard random‐effects (mixed‐effects) model you write:

y
i
j
=
μ
+
τ
j
+
ε
i
j
,
y
ij
​
 =μ+τ
j
​
 +ε
ij
​
 ,
where the
τ
j
τ
j
​
 's are assumed to be independent draws from a distribution, usually

τ
j
∼
N
(
0
,
σ
τ
2
)
.
τ
j
​
 ∼N(0,σ
τ
2
​
 ).
Because
E
(
τ
j
)
=
0
E(τ
j
​
 )=0, we have

E
(
y
i
j
)
=
μ
.
E(y
ij
​
 )=μ.
Under this formulation, the null hypothesis that the zip codes do not differ is equivalent to testing

H
0
:
σ
τ
2
=
0.
H
0
​
 :σ
τ
2
​
 =0.
In other words, if
σ
τ
2
=
0
σ
τ
2
​
 =0, then all the
τ
j
τ
j
​
 's are (with probability 1) 0, meaning every zip code has the same mean
μ
μ. This is why, in a random effects model, you test whether the variance component is zero

 In a standard random‐effects model, the null hypothesis is that the variance of the random intercepts is zero (i.e.
σ
τ
2
=
0
σ
τ
2
​
 =0), which is equivalent to all the
τ
j
τ
j
​
 's being zero. Because this null hypothesis lies on the boundary of the parameter space (variances must be nonnegative), the likelihood ratio test (LRT) statistic doesn't follow a standard chi‐square distribution. Instead, it is asymptotically distributed as a 50:50 mixture of a point mass at zero and a chi‐square with 1 degree of freedom. This “mixture chi-square” LRT is the classical solution for testing whether it makes sense to include a random intercept (i.e. whether there is meaningful between-group variability) in these models.

 The model is parameterized in terms of the variance component
σ
τ
2
σ
τ
2
​
 , not the individual
τ
j
τ
j
​
 's. Testing
H
0
:
σ
τ
2
=
0
H
0
​
 :σ
τ
2
​
 =0 is a single hypothesis about the distribution of the random effects. So while it might sound more intuitive to test “are all the
τ
j
τ
j
​
  exactly 0?”, in the context of a random effects model this is equivalent to testing whether the variance component is zero.


  Below is an example using the likelihood‐ratio test (LRT) to compare a mixed‐effects model (with a random intercept for zip codes) to a simple fixed‐effects model (global mean only). The null hypothesis here is that the variance of the random intercepts (the
t
j
t
j
​
 's) is zero—i.e. that adding zip-code–specific effects does not improve the model.

Because the null lies on the boundary of the parameter space (variance components must be
≥
0
≥0), the usual chi‐square distribution for the LRT does not hold exactly; under the null the LRT is asymptotically distributed as a 50:50 mixture of a point mass at 0 and a chi‑square with 1 degree of freedom. (In practice many people use the mixture approximation, or they resort to a parametric bootstrap.) Here, we’ll use the mixture approximation: for an observed LRT
x
>
0
x>0, the p‑value is approximately

p
≈
0.5
(
1
−
χ
1
2
(
x
)
)
.
p≈0.5(1−χ
1
2
​
 (x)).
If
x
=
0
x=0, we set
p
=
1
p=1.

Explanation
Data Simulation:
We simulate data for 100 zip codes with 20 observations each. Under the null hypothesis, there is no true zip-code effect (all data come from a global distribution with mean 0).
Full Model:
We fit a mixed-effects model using statsmodels’ mixedlm with a random intercept for each zip code. The parameter of interest is the variance component of the random intercepts.
Reduced Model:
We fit a simple OLS model (global mean only), which does not allow for zip-code differences.
Likelihood Ratio Test:
We compute the LRT statistic:
LRT
=
2
(
llf
full
−
llf
reduced
)
.
LRT=2(llf
full
​
 −llf
reduced
​
 ).
Because testing a variance component is on the boundary of the parameter space, we use the mixture chi‑square approximation. For
L
R
T
>
0
LRT>0, the p‑value is approximately:

p
≈
0.5
(
1
−
χ
1
2
(
L
R
T
)
)
.
p≈0.5(1−χ
1
2
​
 (LRT)).
Interpretation:
A significant result (small p‑value) would indicate that including zip-code–specific effects (the
t
j
t
j
​
 's) improves the model, i.e. that not all of them are zero.
This approach directly tests whether the random effects (the zip-code deviations) add explanatory power beyond a single global mean.

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
from scipy.stats import chi2

# -----------------------------
# Simulate example data:
# -----------------------------
np.random.seed(123)
n_zip = 100        # number of zip codes
n_per_zip = 20     # number of observations per zip code

data = []
# Under the null, the true model has no zip-code effect (variance component = 0)
for i in range(n_zip):
    for j in range(n_per_zip):
        # All observations come from the same distribution (global mean 0)
        y = 0 + np.random.normal(0, 2)  # global mean 0, error SD = 2
        data.append((i, y))

df = pd.DataFrame(data, columns=['zip', 'y'])

# -----------------------------
# Fit the full mixed-effects model:
# -----------------------------
# Model with random intercepts for each zip code.
full_model = smf.mixedlm("y ~ 1", data=df, groups=df["zip"])
full_res = full_model.fit()
print("Full model (random intercepts) summary:")
print(full_res.summary())

# -----------------------------
# Fit the reduced model (no random effects):
# -----------------------------
# This is equivalent to an OLS regression with a single intercept.
reduced_model = smf.ols("y ~ 1", data=df)
reduced_res = reduced_model.fit()
print("\nReduced model (global mean only) summary:")
print(reduced_res.summary())

# -----------------------------
# Likelihood Ratio Test:
# -----------------------------
# LRT statistic: 2*(logL_full - logL_reduced)
LRT = 2 * (full_res.llf - reduced_res.llf)
print("\nLikelihood Ratio Test statistic:", LRT)

# Under H0, the LRT follows a 50:50 mixture: 0.5*delta(0) + 0.5*chi2(1)
if LRT > 0:
    p_value = 0.5 * (1 - chi2.cdf(LRT, 1))
else:
    p_value = 1.0
print("P-value for testing that the random effects variance is zero:", p_value)
